<a href="https://colab.research.google.com/github/wldnjsrnjs/EpigenAI/blob/main/EpigenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# 데이터 만들기
data = pd.DataFrame({
    'Lifestyle_Factor': ['Sleep_Hours', 'Sleep_Hours', 'Stress_Level', 'Stress_Level', 'Diet_Quality', 'Diet_Quality', 'Exercise_Minutes', 'Exercise_Minutes'],
    'Value': ['6', '8', 'High', 'Low', 'Poor', 'Good', '30', '0'],
    'Epigenetic_Mechanism': ['DNA Methylation (Increased)', 'DNA Methylation (Balanced)', 'Histone Modification (Dysregulated)', 'Histone Modification (Stable)', 'DNA Methylation (Increased)', 'DNA Methylation (Balanced)', 'DNA Methylation (Decreased)', 'DNA Methylation (Increased)'],
    'Health_Risk_Score': [3.5, 1.5, 4.0, 1.5, 4.5, 2.0, 1.5, 3.5],
    'Alzheimer_Risk': [0.4, 0.1, 0.3, 0.1, 0.2, 0.1, 0.1, 0.2],
    'Cardiovascular_Risk': [0.3, 0.1, 0.5, 0.1, 0.4, 0.1, 0.1, 0.3],
    'Diabetes_Risk': [0.2, 0.1, 0.2, 0.1, 0.5, 0.1, 0.1, 0.3],
    'Source': ['Search Result 1: DNA 메틸화 패턴 변화로 노화 가속', 'Search Result 1: 적정 수면 시 메틸화 균형', 'Search Result 3: 스트레스가 후성유전적 변화 유발', 'Search Result 3: 스트레스 감소 시 안정성 유지', 'Search Result 5: 잘못된 식습관이 메틸화 변화 유발', 'Search Result 5: 균형 잡힌 식단이 안정성 유지', 'Search Result 2: 적절한 운동이 메틸화 감소', 'Search Result 2: 운동 부족이 메틸화 변화 유발']
})

# CSV 파일로 저장
data.to_csv('epigen_data.csv', index=False)
print("데이터 파일이 저장되었습니다!")


데이터 파일이 저장되었습니다!


In [ ]:
from flask import Flask, request, render_template_string
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import os
from transformers import pipeline

app = Flask(__name__)

# 데이터베이스 로드
data = pd.read_csv('epigen_data.csv')

# 인공지능 API 로드 (Hugging Face)
generator = pipeline('text-generation', model='gpt2')

def calculate_risk(user_data):
    total_risk = 0
    alzheimer_risk = 0.1
    cardio_risk = 0.1
    diabetes_risk = 0.1
    count = 0

    for factor, value in user_data.items():
        if factor != 'age_group' and factor != 'date':
            matching_row = data[(data['Lifestyle_Factor'] == factor) & (data['Value'] == value)]
            if not matching_row.empty:
                total_risk += matching_row['Health_Risk_Score'].iloc[0]
                alzheimer_risk = max(alzheimer_risk, matching_row['Alzheimer_Risk'].iloc[0])
                cardio_risk = max(cardio_risk, matching_row['Cardiovascular_Risk'].iloc[0])
                diabetes_risk = max(diabetes_risk, matching_row['Diabetes_Risk'].iloc[0])
                count += 1

    return total_risk / max(1, count), alzheimer_risk, cardio_risk, diabetes_risk

def generate_personalized_advice(risk_score, user_input, age_group):
    base_prompt = f"User has a health risk score of {risk_score} (1-5, higher is worse). They are {'over 60 years old' if age_group == 'over_60' else 'under 60'}. "
    if user_input:
        base_prompt += f"They mentioned: {user_input}. "
    base_prompt += "Provide personalized health advice in a friendly tone to help prevent aging-related diseases."
    response = generator(base_prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
    return response

def generate_report(risk_score, alzheimer_risk, cardio_risk, diabetes_risk, advice, user_data):
    report = f"""
    EpigenAI 건강 분석 리포트
    -----------------------
    날짜: {user_data['date']}
    건강 위험도 점수: {risk_score:.2f} (1~5, 높을수록 위험)
    노화 질병 위험도:
    - 알츠하이머: {alzheimer_risk*100:.0f}% 수준
    - 심혈관 질환: {cardio_risk*100:.0f}% 수준
    - 당뇨병: {diabetes_risk*100:.0f}% 수준
    사용자 입력:
    - 연령대: {user_data['age_group']}
    - 수면 시간: {user_data['sleep']}
    - 스트레스 수준: {user_data['stress']}
    - 식단 품질: {user_data['diet']}
    - 운동 시간: {user_data['exercise']}
    건강 조언:
    {advice}
    """
    report_file = f"report_{os.urandom(4).hex()}.txt"
    with open(report_file, "w") as f:
        f.write(report)
    return "리포트가 생성되었습니다. (실제 PDF 기능은 개발 중입니다.)"

def simulate_improvement(user_data, factor, improved_value):
    improved_data = user_data.copy()
    improved_data[factor] = improved_value
    risk, _, _, _ = calculate_risk(improved_data)
    return risk

def save_user_data(user_data, risk_score, advice):
    conn = sqlite3.connect('epigenai_users.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users
                      (id INTEGER PRIMARY KEY AUTOINCREMENT, age_group TEXT, sleep TEXT, stress TEXT, diet TEXT, exercise TEXT, risk_score REAL, advice TEXT, date TEXT, timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')
    cursor.execute('INSERT INTO users (age_group, sleep, stress, diet, exercise, risk_score, advice, date) VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
                   (user_data['age_group'], user_data['sleep'], user_data['stress'], user_data['diet'], user_data['exercise'], risk_score, advice, user_data['date']))
    conn.commit()
    conn.close()

def check_user_records():
    conn = sqlite3.connect('epigenai_users.db')
    cursor = conn.cursor()
    cursor.execute('SELECT DISTINCT date FROM users')
    dates = cursor.fetchall()
    conn.close()
    return len(dates) >= 7

def calculate_weekly_summary():
    conn = sqlite3.connect('epigenai_users.db')
    cursor = conn.cursor()
    cursor.execute('SELECT risk_score, date FROM users ORDER BY timestamp DESC LIMIT 7')
    records = cursor.fetchall()
    conn.close()
    if len(records) < 7:
        return None, None, None, None
    risk_scores = [r[0] for r in records]
    avg_risk = sum(risk_scores) / len(risk_scores)
    trend = "개선됨" if risk_scores[0] < risk_scores[-1] else "악화됨" if risk_scores[0] > risk_scores[-1] else "유사함"
    return avg_risk, trend, min(risk_scores), max(risk_scores)

@app.route('/', methods=['GET', 'POST'])
def home():
    if request.method == 'POST':
        age_group = request.form.get('age_group', 'under_60')
        sleep_hours = request.form.get('sleep', '6')
        stress_level = request.form.get('stress', 'Low')
        diet_quality = request.form.get('diet', 'Good')
        exercise_minutes = request.form.get('exercise', '30')
        date = request.form.get('date', datetime.now().strftime('%Y-%m-%d'))
        user_input = request.form.get('user_comment', '')

        user_data = {
            'age_group': age_group,
            'sleep': sleep_hours,
            'stress': stress_level,
            'diet': diet_quality,
            'exercise': exercise_minutes,
            'date': date
        }
        risk, alzheimer_risk, cardio_risk, diabetes_risk = calculate_risk({
            'age_group': age_group,
            'Sleep_Hours': sleep_hours,
            'Stress_Level': stress_level,
            'Diet_Quality': diet_quality,
            'Exercise_Minutes': exercise_minutes
        })
        advice = generate_personalized_advice(risk, user_input, age_group)
        save_user_data(user_data, risk, advice)

        return render_template_string('''
            <html>
            <head>
                <title>EpigenAI 입력 완료</title>
                <style>
                    body { font-family: Arial, sans-serif; font-size: 20px; text-align: center; background-color: #f0f0f0; }
                    h1 { color: #2c3e50; font-size: 28px; }
                    p { margin: 20px; line-height: 1.6; }
                    .result-box { background-color: white; border-radius: 10px; padding: 20px; margin: 30px auto; width: 80%; box-shadow: 0 0 10px rgba(0,0,0,0.1); }
                    .button { display: inline-block; padding: 10px 20px; background-color: #3498db; color: white; text-decoration: none; border-radius: 5px; font-size: 18px; margin-top: 20px; }
                </style>
            </head>
            <body>
                <h1>EpigenAI: 데이터 입력 완료</h1>
                <div class="result-box">
                    <p>입력하신 데이터가 저장되었습니다. 날짜: {{ user_data['date'] }}</p>
                    <p>최소 7일의 기록이 모이면 건강 분석 결과를 확인할 수 있습니다.</p>
                    <a href="/" class="button">추가 데이터 입력</a>
                    <a href="/history" class="button">나의 기록 보기</a>
                    {% if has_enough_records %}
                        <a href="/results" class="button">주간 결과 분석</a>
                    {% endif %}
                    <a href="/education" class="button">후성유전학 교육</a>
                </div>
            </body>
            </html>
        ''', user_data=user_data, has_enough_records=check_user_records())
    return '''
        <html>
        <head>
            <title>EpigenAI</title>
            <style>
                body { font-family: Arial, sans-serif; font-size: 20px; text-align: center; background-color: #f0f0f0; }
                h1 { color: #2c3e50; font-size: 28px; }
                p { margin: 20px; line-height: 1.6; }
                form { background-color: white; border-radius: 10px; padding: 20px; margin: 30px auto; width: 80%; box-shadow: 0 0 10px rgba(0,0,0,0.1); }
                label { display: inline-block; width: 200px; text-align: right; margin-right: 10px; }
                select, textarea, input[type="date"] { font-size: 18px; padding: 5px; width: 300px; }
                input[type="submit"] { padding: 10px 20px; background-color: #3498db; color: white; border: none; border-radius: 5px; font-size: 18px; cursor: pointer; }
                input[type="submit"]:hover { background-color: #2980b9; }
            </style>
        </head>
        <body>
            <h1>EpigenAI: 후성유전학 기반 건강습관 분석</h1>
            <p>고령화 사회에서의 노화 질병 예방을 위해 생활습관 데이터를 입력하세요. 최소 7일의 기록이 모이면 건강 분석 결과를 확인할 수 있습니다.</p>
            <form method="post">
                <label>연령대:</label>
                <select name="age_group">
                    <option value="under_60">60세 미만</option>
                    <option value="over_60">60세 이상</option>
                </select><br><br>
                <label>날짜:</label>
                <input type="date" name="date" value="{{ today }}" required><br><br>
                <label>수면 시간 (시간):</label>
                <select name="sleep">
                    <option value="6">6시간</option>
                    <option value="8">8시간</option>
                </select><br><br>
                <label>스트레스 수준:</label>
                <select name="stress">
                    <option value="Low">낮음</option>
                    <option value="High">높음</option>
                </select><br><br>
                <label>식단 품질:</label>
                <select name="diet">
                    <option value="Good">좋음</option>
                    <option value="Poor">나쁨</option>
                </select><br><br>
                <label>운동 시간 (분):</label>
                <select name="exercise">
                    <option value="30">30분</option>
                    <option value="0">0분</option>
                </select><br><br>
                <label>추가 의견 (선택 사항):</label>
                <textarea name="user_comment" rows="3" cols="30" placeholder="건강 상태나 궁금한 점을 적어주세요."></textarea><br><br>
                <input type="submit" value="데이터 저장">
            </form>
        </body>
        </html>
    '''.replace('{{ today }}', datetime.now().strftime('%Y-%m-%d'))

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# 간단한 HTML 파일 만들기 (EpigenAI 앱의 홈페이지)
html_content = """
<html>
<head>
    <title>EpigenAI</title>
    <style>
        body { font-family: Arial, sans-serif; font-size: 20px; text-align: center; background-color: #f0f0f0; }
        h1 { color: #2c3e50; font-size: 28px; }
        p { margin: 20px; line-height: 1.6; }
        form { background-color: white; border-radius: 10px; padding: 20px; margin: 30px auto; width: 80%; box-shadow: 0 0 10px rgba(0,0,0,0.1); }
        label { display: inline-block; width: 200px; text-align: right; margin-right: 10px; }
        select, textarea, input[type="date"] { font-size: 18px; padding: 5px; width: 300px; }
        input[type="submit"] { padding: 10px 20px; background-color: #3498db; color: white; border: none; border-radius: 5px; font-size: 18px; cursor: pointer; }
        input[type="submit"]:hover { background-color: #2980b9; }
    </style>
</head>
<body>
    <h1>EpigenAI: 후성유전학 기반 건강습관 분석</h1>
    <p>고령화 사회에서의 노화 질병 예방을 위해 생활습관 데이터를 입력하세요. 최소 7일의 기록이 모이면 건강 분석 결과를 확인할 수 있습니다.</p>
    <form method="post">
        <label>연령대:</label>
        <select name="age_group">
            <option value="under_60">60세 미만</option>
            <option value="over_60">60세 이상</option>
        </select><br><br>
        <label>날짜:</label>
        <input type="date" name="date" value="2025-05-27" required><br><br>
        <label>수면 시간 (시간):</label>
        <select name="sleep">
            <option value="6">6시간</option>
            <option value="8">8시간</option>
        </select><br><br>
        <label>스트레스 수준:</label>
        <select name="stress">
            <option value="Low">낮음</option>
            <option value="High">높음</option>
        </select><br><br>
        <label>식단 품질:</label>
        <select name="diet">
            <option value="Good">좋음</option>
            <option value="Poor">나쁨</option>
        </select><br><br>
        <label>운동 시간 (분):</label>
        <select name="exercise">
            <option value="30">30분</option>
            <option value="0">0분</option>
        </select><br><br>
        <label>추가 의견 (선택 사항):</label>
        <textarea name="user_comment" rows="3" cols="30" placeholder="건강 상태나 궁금한 점을 적어주세요."></textarea><br><br>
        <input type="submit" value="데이터 저장">
    </form>
</body>
</html>
"""
# HTML 파일 저장
with open('index.html', 'w') as f:
    f.write(html_content)
print("HTML 파일이 저장되었습니다!")
